In [1]:
import os 
import sys
sys.path.append(os.path.join(os.getcwd(), 'src/'))

In [2]:
sys.path

['/Users/Bryan.Liu/oce-ecomm-abv-calculation',
 '/Users/Bryan.Liu/.pyenv/versions/3.9.10/lib/python39.zip',
 '/Users/Bryan.Liu/.pyenv/versions/3.9.10/lib/python3.9',
 '/Users/Bryan.Liu/.pyenv/versions/3.9.10/lib/python3.9/lib-dynload',
 '',
 '/Users/Bryan.Liu/oce-ecomm-abv-calculation/.venv/lib/python3.9/site-packages',
 '/Users/Bryan.Liu/oce-ecomm-abv-calculation/src/']

In [4]:
from oce_ecomm_abv_calculation.datasets.ecommerce_dataset import ECommerceDataset
import pydantic

class UCIOnlineRetailIIDataSet(ECommerceDataset):
    pass